In [2]:
# Parte 1 - creo dataframe
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
from MJP.majority_portfolio_utilities_TV import * 

FILENAME = 'monthly_data.csv'
factors_df=pd.DataFrame({
    'factors': ['ag','beta','bm','cumret','dolvol6','gp','ill6','ns','size','volatility','acc'],
    'signs': [  -1.,  1.,   1.,    1.,     -1.,     1.,    1.,  -1.,  -1.,   -1.,   -1.],
    'wsigns':[  -1.,  1.,   1.,    1.,     -1.,     1.,    1.,  -1.,  -1.,   -1.,   -1.]
})

DATA_INIZIO = '2000-01-01'      # In futuro considerare periodi bull e bear
DATA_FINE = '2023-12-31'        # Quindi utilizzare sub periodi

df = create_df(FILENAME, factors_df['factors'].tolist(), DATA_INIZIO, DATA_FINE) # funzione sua
df['date']=pd.to_datetime(df['date']) 
print(f"DataFrame caricato: {df.shape[0]:,} righe, {df.shape[1]} colonne")
print(df.columns.tolist())

# PARTE 2 - NORMALIZZO CON QUANTILE NORMALIZER

from sklearn.preprocessing import QuantileTransformer

factors_to_normalize = ['ns', 'size', 'ag', 'gp', 'acc', 'bm', 'ill6', 'volatility', 'beta', 'cumret', 'dolvol6']

# Elimina righe con NaN solo nelle colonne da normalizzare
df_clean = df.dropna(subset=factors_to_normalize)

# Applica Quantile normalization solo ai fattori
scaler = QuantileTransformer(output_distribution='uniform', random_state=0)
df_norm = df_clean.copy()
df_norm[factors_to_normalize] = scaler.fit_transform(df_clean[factors_to_normalize])

print("✅ df normalizzato su fattori:")
df_norm.head()

DataFrame caricato: 1,099,846 righe, 19 colonne
['PERMNO', 'date', 'RET', 'ns', 'me', 'size', 'me_lag', 'ag', 'gp', 'acc', 'bm', 'RF', 'Mkt_RF', 'RET_RF', 'cumret', 'dolvol6', 'ill6', 'volatility', 'beta']
✅ df normalizzato su fattori:


,PERMNO,date,RET,ns,me,size,me_lag,ag,gp,acc,bm,RF,Mkt_RF,RET_RF,cumret,dolvol6,ill6,volatility,beta
0,10001,2000-01-31,-0.044118,0.552123,19.90625,0.089730,20.82500,0.429226,0.186763,0.098351,0.751966,0.0041,-0.0474,-0.048218,0.406036,0.051494,0.868132,0.039530,0.100142
1,10001,2000-02-29,0.015385,0.552123,20.21250,0.090958,19.90625,0.429226,0.186763,0.098351,0.746500,0.0043,0.0245,0.011085,0.449992,0.047740,0.875944,0.018688,0.100379
2,10001,2000-03-31,-0.015758,0.619632,19.71200,0.088813,20.21250,0.429226,0.186763,0.098351,0.754909,0.0047,0.0520,-0.020458,0.452531,0.047380,0.864058,0.018764,0.102204
3,10001,2000-04-30,0.011719,0.619632,19.94300,0.089905,19.71200,0.429226,0.186763,0.098351,0.751315,0.0046,-0.0640,0.007119,0.424543,0.044751,0.822586,0.018568,0.096001
4,10001,2000-05-31,-0.023166,0.619632,19.48100,0.087670,19.94300,0.429226,0.186763,0.098351,0.759141,0.0050,-0.0442,-0.028166,0.455732,0.040165,0.850747,0.018772,0.100983


In [3]:
# Parte 3 - inizializzazione della configurazione

# *** ATTENZIONE *** --> PRIMA MODIFICA, HO MESSO TRUE A WEIGHT

remove_outliers = False
inf, sup = None, None
MJ_configuration={
    'K' : 12, # holding_periods in months
    'lag' : 5, # 5 for june reallocation
    'factors' : factors_df['factors'].to_list(),
    'num_port' : 10, # number of portfolios
    'num_cat' : 6, # number of categorical variables to use per factor in mj #'num_cat': 6
    'weighting': True,
    'verbose': True,
    'n_jobs':-1,
    'mj_window':1,
    'method' : 'majority', # mean_rank, majority, 75q, 90q, lex, dlex
    'rolling_method':'profile',  ###'rank','vote','profile'
    'treat_na_mj':'median',
    'remove_outliers':remove_outliers,
    'inclusive':True,
    'fix_signs':True, # True: tieni fisso il numero di votanti ed il loro segno
    'all_voters_not_nan_on_reallocation':True,
    #### PARAMETRI UTILIZZATI SOLO SE fix_signs=False ####
    'min_voters':5,    
    'voting_window':6,
    'sign_voting_window':12,
    'p_threshold' : 0.1, 
    'delta_utility' : 0,
    'eliminations': 1,
    'players_batch_size': 5,
    'small': True
}
if remove_outliers:
    MJ_configuration['outliers']=[inf,sup]

if MJ_configuration['weighting']:
    MJ_configuration['default_voters']=factors_df['factors'].to_list()
    MJ_configuration['default_signs']=factors_df['wsigns'].to_list()
    compute='wmj'
elif not MJ_configuration['weighting']:
    MJ_configuration['default_voters']=factors_df['factors'].to_list()
    MJ_configuration['default_signs']=factors_df['signs'].to_list()
    compute='mj'

factors=factors_df['factors'].to_list() # salva separatamente la lista dei nomi dei fattori 

In [4]:
# Parte 4 - creazione portafogli single criteria e MJ

### COMPUTE SINGLE FACTOR STRATEGIES
portfolios, weighted_portfolios, portfolios_stock_reallocation = compute_factor_strategies(df_norm, MJ_configuration)
        # '2020-06-30': [list of tickers]       # Con pesi, uguale ma 'AAPL': 0.1           # 'AAPL': 2020-06-30': 'P3', '2021-06-30': 'P5'}
        # '2021-06-30': [list of tickers]
        # _ perché non considero weighted_portfolios al momento

### COMPUTE EQUAL WEIGHTED MJ STRATEGY
MJ_configuration['weighting']=False
MJ_portfolios, \
mj_voters, \
MJ_portfolios_stock_reallocation = compute_MJ_portfolio_strategy(df_norm, MJ_configuration)

portfolios['mj'] = MJ_portfolios['mj']
portfolios_stock_reallocation['mj'] = {}
portfolios_stock_reallocation['mj']['EW_turnover'] = MJ_portfolios_stock_reallocation['mj']

### COMPUTE VALUE WEIGHTED MJ STRATEGY
MJ_configuration['weighting']=True
MJ_weighted_portfolios, \
mj_voters, \
MJ_weighted_portfolios_stock_reallocation = compute_MJ_portfolio_strategy(df_norm, MJ_configuration)

weighted_portfolios['wmj'] = MJ_weighted_portfolios['wmj']
portfolios_stock_reallocation['wmj'] = {}
portfolios_stock_reallocation['wmj']['VW_turnover'] = MJ_weighted_portfolios_stock_reallocation['wmj']

'''
OUTPUT: 
portfolios = {
  'ns': DataFrame con port1...port10 costruiti su ns,
  'beta': ...,
  'bm': ...,
  ...
  'mj': DataFrame con port1...port10 creati con MJ
}
'''


"\nOUTPUT: \nportfolios = {\n  'ns': DataFrame con port1...port10 costruiti su ns,\n  'beta': ...,\n  'bm': ...,\n  ...\n  'mj': DataFrame con port1...port10 creati con MJ\n}\n"

In [5]:
{ "cells": [ { "cell_type": "code", "metadata": { "language": "python" }, "source": [ "import importlib, traceback", "import MJP.majority_portfolio_utilities_TV as mpu", "import ahpy", "", "print('Reloading module MJP.majority_portfolio_utilities_TV...')", "importlib.reload(mpu)", "print('ahpy version:', getattr(ahpy, 'version', 'unknown'))", "", "try:", " AHP_portfolios, AHP_portfolios_stock_reallocation = mpu.compute_mcdm_rolling_strategy(df_norm, 'ahp', MJ_configuration, weighting=False)", " print('compute_mcdm_rolling_strategy completed successfully')", "except Exception as e:", " # Print full traceback and repr to capture any diagnostic message we raised in the module\n", " traceback.print_exc()", " print('\nException repr:', repr(e))" ] } ] }

{'cells': [{'cell_type': 'code',
   'metadata': {'language': 'python'},
   'source': ['import importlib, traceback',
    'import MJP.majority_portfolio_utilities_TV as mpu',
    'import ahpy',
    '',
    "print('Reloading module MJP.majority_portfolio_utilities_TV...')",
    'importlib.reload(mpu)',
    "print('ahpy version:', getattr(ahpy, 'version', 'unknown'))",
    '',
    'try:',
    " AHP_portfolios, AHP_portfolios_stock_reallocation = mpu.compute_mcdm_rolling_strategy(df_norm, 'ahp', MJ_configuration, weighting=False)",
    " print('compute_mcdm_rolling_strategy completed successfully')",
    'except Exception as e:',
    ' # Print full traceback and repr to capture any diagnostic message we raised in the module\n',
    ' traceback.print_exc()',
    " print('\nException repr:', repr(e))"]}]}

In [8]:
# Ricordarsi che ora tutti i pesi sono uguali ad 1

import ahpy # per ahp
from pyDecision.algorithm import electre_iii # per electre
from tqdm import tqdm

criteria_pairwise = {}
for i in range(len(factors)):
    for j in range(i+1, len(factors)):
        criteria_pairwise[(factors[i], factors[j])] = 1.0

def compute_ahp_rolling_strategy(df: pd.DataFrame, configuration: dict, criteria_pairwise: dict, weighting: bool = False, verbose: bool = False):
    df = df.copy()
    df['medate'] = df['date'] + MonthEnd(0)

    K = configuration['K']
    lag = configuration['lag']
    num_port = configuration['num_port']

    factors = sorted(configuration['factors'])  # Assumiamo lista fattori ordinata
    types = []
    if 'default_signs' in configuration:
        signs = configuration['default_signs']
        types = [signs[configuration['factors'].index(f)] for f in factors]
    else:
        types = [1]*len(factors)

    # Costruisci Compare con ahpy
    try:
        criteria_compare = ahpy.Compare('Criteria', criteria_pairwise, precision=3)
        weights_dict = criteria_compare.target_weights
        weights_vector = np.array([weights_dict[f] for f in factors])
        if verbose:
            print("AHP Pesi criteri:", weights_dict)
            print("AHP Consistency Ratio:", criteria_compare.consistency_ratio)
    except AssertionError as e:
        raise ValueError(f"Errore nella costruzione AHP: {e}")

    ascending = False  # in AHP punteggio alto è meglio

    reallocation_dates = []
    date_start = df['medate'].min() + MonthEnd(lag)
    date_end = df['medate'].max()
    while date_start + MonthEnd(K) <= date_end:
        reallocation_dates.append(date_start)
        date_start += MonthEnd(K)

    returns_dict = {f'port{i+1}': [] for i in range(num_port)}
    returns_dict['long_short'] = []

    turnover_values = {f'port{i+1}': [] for i in range(num_port)}
    reallocation = {f'port{i+1}': {} for i in range(num_port)}
    prev_portfolios = {f'port{i+1}': None for i in range(num_port)}

    for date in reallocation_dates:
        df_now = df[df['medate'] == date].dropna(subset=factors).copy()
        if df_now.empty or len(df_now) < num_port:
            continue

        matrix = df_now[factors].values

        assert matrix.shape[1] == len(weights_vector), "Dimensione matrice fattori diversa da pesi AHP"

        scores = matrix.dot(weights_vector)

        df_now['score'] = scores
        df_now = df_now.sort_values('score', ascending=ascending).reset_index(drop=True)

        labels = [f'port{i+1}' for i in reversed(range(num_port))]
        df_now['portfolio'] = pd.qcut(df_now.index, q=num_port, labels=labels)

        period_end = date + MonthEnd(K)
        df_hold = df[(df['medate'] > date) & (df['medate'] <= period_end)]

        port_permnos = {}
        for port in labels:
            tickers = df_now[df_now['portfolio'] == port]['PERMNO'].tolist()
            port_permnos[port] = tickers
            reallocation[port][date] = tickers
            df_port = df_hold[df_hold['PERMNO'].isin(tickers)].copy()

            if weighting:
                df_port['wret'] = df_port['RET_RF'] * df_port['me_lag']
                ret = df_port.groupby('medate').apply(lambda x: x['wret'].sum() / x['me_lag'].sum())
            else:
                ret = df_port.groupby('medate')['RET_RF'].mean()

            returns_dict[port].append(ret)

        for port in labels:
            curr_holdings = set(port_permnos[port])
            if prev_portfolios[port] is not None:
                prev_holdings = set(prev_portfolios[port])
                intersection = len(prev_holdings & curr_holdings)
                total_unique = max(len(prev_holdings), len(curr_holdings))
                turnover_rate = 1 - (intersection / total_unique) if total_unique > 0 else 0.0
            else:
                turnover_rate = np.nan
            turnover_values[port].append(turnover_rate)
            prev_portfolios[port] = port_permnos[port]

        long = df_hold[df_hold['PERMNO'].isin(port_permnos['port10'])].copy()
        short = df_hold[df_hold['PERMNO'].isin(port_permnos['port1'])].copy()

        if weighting:
            long['wret'] = long['RET_RF'] * long['me_lag']
            short['wret'] = short['RET_RF'] * short['me_lag']
            ret_long = long.groupby('medate').apply(lambda x: x['wret'].sum() / x['me_lag'].sum())
            ret_short = short.groupby('medate').apply(lambda x: x['wret'].sum() / x['me_lag'].sum())
        else:
            ret_long = long.groupby('medate')['RET_RF'].mean()
            ret_short = short.groupby('medate')['RET_RF'].mean()

        ls_ret = ret_long - ret_short
        returns_dict['long_short'].append(ls_ret)

    for key in returns_dict:
        returns_dict[key] = pd.concat(returns_dict[key]).sort_index()

    portfolios_df = pd.DataFrame(returns_dict).dropna()

    avg_turnover = {
        port: np.nanmean(values) if values else np.nan
        for port, values in turnover_values.items()
    }

    turnover_key = 'VW_turnover' if weighting else 'EW_turnover'
    turnover_df = pd.DataFrame({
        'portfolio': [int(p.replace('port', '')) for p in avg_turnover.keys()],
        turnover_key: list(avg_turnover.values())
    })

    portfolios_stock_reallocation = {
        turnover_key: turnover_df,
        'reallocation': reallocation
    }

    return portfolios_df, portfolios_stock_reallocation

def compute_electre_rolling_strategy(df: pd.DataFrame, configuration: dict, weighting: bool = False, n_jobs: int = -1):
    """
    Rolling strategy ELECTRE III ottimizzata con parallelismo per finestre di ribilanciamento.
    """
    df = df.copy()
    df['medate'] = df['date'] + MonthEnd(0)

    # Configurazione base
    K = configuration['K']
    lag = configuration['lag']
    num_port = configuration['num_port']
    factors = sorted(configuration['factors'])

    # Tipologia criteri (+1 beneficio, -1 costo)
    if 'default_signs' in configuration:
        signs = configuration['default_signs']
        types_dict = {k: signs[i] for i, k in enumerate(configuration['factors'])}
        types = [types_dict[f] for f in factors]
    else:
        types = [1] * len(factors)

    weights = np.ones(len(factors)) / len(factors)
    indifference = configuration.get('indifference', [0.1]*len(factors))
    preference = configuration.get('preference', [0.3]*len(factors))
    veto = configuration.get('veto', [0.5]*len(factors))
    verbose = configuration.get('verbose', False)

    # Date di ribilanciamento
    reallocation_dates = []
    date_start = df['medate'].min() + MonthEnd(lag)
    date_end = df['medate'].max()
    while date_start + MonthEnd(K) <= date_end:
        reallocation_dates.append(date_start)
        date_start += MonthEnd(K)

    # Funzione per processare una singola finestra
    def process_window(date):
        df_now = df[df['medate'] == date].dropna(subset=factors)
        if df_now.empty or len(df_now) < num_port:
            return None  # Ignora finestre vuote

        matrix = df_now[factors].values
        try:
            scores = electre_iii(matrix, weights, types, indifference, preference, veto)
            scores = np.max(scores) - np.array(scores)  # ranking invertito
        except:
            return None

        df_now = df_now.copy()
        df_now['score'] = scores
        df_now = df_now.sort_values('score', ascending=False).reset_index(drop=True)

        labels = [f'port{i+1}' for i in reversed(range(num_port))]
        df_now['portfolio'] = pd.qcut(df_now.index, q=num_port, labels=labels)

        period_end = date + MonthEnd(K)
        df_hold = df[(df['medate'] > date) & (df['medate'] <= period_end)]

        port_returns = {}
        port_permnos = {}
        for port in labels:
            tickers = df_now[df_now['portfolio'] == port]['PERMNO'].tolist()
            port_permnos[port] = tickers
            df_port = df_hold[df_hold['PERMNO'].isin(tickers)].copy()
            if weighting:
                df_port['wret'] = df_port['RET_RF'] * df_port['me_lag']
                ret = df_port.groupby('medate').apply(lambda x: x['wret'].sum()/x['me_lag'].sum())
            else:
                ret = df_port.groupby('medate')['RET_RF'].mean()
            port_returns[port] = ret

        # Long-short
        long = df_hold[df_hold['PERMNO'].isin(port_permnos['port10'])].copy()
        short = df_hold[df_hold['PERMNO'].isin(port_permnos['port1'])].copy()
        if weighting:
            long['wret'] = long['RET_RF'] * long['me_lag']
            short['wret'] = short['RET_RF'] * short['me_lag']
            ls_ret = long.groupby('medate').apply(lambda x: x['wret'].sum()/x['me_lag'].sum()) - \
                     short.groupby('medate').apply(lambda x: x['wret'].sum()/x['me_lag'].sum())
        else:
            ls_ret = long.groupby('medate')['RET_RF'].mean() - short.groupby('medate')['RET_RF'].mean()

        return {'date': date, 'port_returns': port_returns, 'long_short': ls_ret, 'port_permnos': port_permnos}

    # Parallel processing
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_window)(d) for d in tqdm(reallocation_dates, desc="ELECTRE rolling progress", ncols=90)
    )

    # --- Combina risultati ---
    returns_dict = {f'port{i+1}': [] for i in range(num_port)}
    returns_dict['long_short'] = []
    turnover_values = {f'port{i+1}': [] for i in range(num_port)}
    reallocation = {f'port{i+1}': {} for i in range(num_port)}
    prev_portfolios = {f'port{i+1}': None for i in range(num_port)}

    for res in results:
        if res is None:
            continue
        date = res['date']
        for port, ret in res['port_returns'].items():
            returns_dict[port].append(ret)
            reallocation[port][date] = res['port_permnos'][port]

        returns_dict['long_short'].append(res['long_short'])

        # Calcolo turnover
        for port in res['port_permnos']:
            curr_holdings = set(res['port_permnos'][port])
            if prev_portfolios[port] is not None:
                prev_holdings = set(prev_portfolios[port])
                intersection = len(prev_holdings & curr_holdings)
                total_unique = max(len(prev_holdings), len(curr_holdings))
                turnover_rate = 1 - (intersection/total_unique) if total_unique>0 else 0
            else:
                turnover_rate = np.nan
            turnover_values[port].append(turnover_rate)
            prev_portfolios[port] = res['port_permnos'][port]

    # --- Unisci i rendimenti ---
    for key in returns_dict:
        returns_dict[key] = pd.concat(returns_dict[key]).sort_index()

    portfolios_df = pd.DataFrame(returns_dict).dropna()

    avg_turnover = {port: np.nanmean(values) if values else np.nan for port, values in turnover_values.items()}
    turnover_key = 'VW_turnover' if weighting else 'EW_turnover'
    turnover_df = pd.DataFrame({
        'portfolio':[int(p.replace('port','')) for p in avg_turnover.keys()],
        turnover_key:list(avg_turnover.values())
    })

    portfolios_stock_reallocation = {'reallocation': reallocation, turnover_key: turnover_df}

    if verbose:
        print("\nELECTRE rolling strategy completata.")

    return portfolios_df, portfolios_stock_reallocation






#AHP_portfolios, AHP_portfolios_stock_reallocation = compute_ahp_rolling_strategy(df_norm, MJ_configuration, criteria_pairwise, weighting=False, verbose=True)
ELECTRE_portfolios, ELECTRE_portfolios_stock_reallocation = compute_electre_rolling_strategy(df_norm, MJ_configuration, weighting=False, n_jobs=-1)


#portfolios['ahp'] = AHP_portfolios
portfolios['electre'] = ELECTRE_portfolios

#portfolios_stock_reallocation['ahp'] = AHP_portfolios_stock_reallocation
portfolios_stock_reallocation['electre'] = ELECTRE_portfolios_stock_reallocation

### COMPUTE VW MCDM STRATEGY
#AHP_portfolios_weighted, AHP_portfolios_stock_reallocation_weighted = compute_ahp_rolling_strategy(df_norm, MJ_configuration, criteria_pairwise, weighting=True, verbose=True)
ELECTRE_portfolios_weighted, ELECTRE_portfolios_stock_reallocation_weighted = compute_electre_rolling_strategy(df_norm, MJ_configuration, weighting=True, n_jobs=-1)

# inserisco VW mcdm in portfolios e reallocation
#weighted_portfolios['wahp'] = AHP_portfolios_weighted
#portfolios_stock_reallocation['wahp'] = AHP_portfolios_stock_reallocation_weighted
weighted_portfolios['welectre'] = ELECTRE_portfolios_weighted
portfolios_stock_reallocation['welectre'] = ELECTRE_portfolios_stock_reallocation_weighted

# Il valore "AHP Consistency Ratio: 0.0" indica che la matrice di confronto a coppie è perfettamente consistente, ovvero non presenta incongruenze nei giudizi espressi.


ELECTRE rolling progress:  35%|█████████▋                  | 8/23 [00:14<00:00, 50.01it/s]

KeyboardInterrupt: 

In [ ]:
# ========================== FACCIO I PORTAFOGLI ==========================

# PARTE 6 - COMPUTE EW-VW MCDM STRATEGY

TOPSIS_portfolios, TOPSIS_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'topsis', MJ_configuration, weighting=False)
VIKOR_portfolios, VIKOR_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'vikor', MJ_configuration, weighting=False)
PROMETHEE_portfolios, PROMETHEE_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'promethee', MJ_configuration, weighting=False)
#AHP_portfolios, AHP_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'ahp', MJ_configuration, weighting=False)


# inserisco EW mcdm in portfolios e reallocation
portfolios['topsis'] = TOPSIS_portfolios # gia' df
portfolios['vikor'] = VIKOR_portfolios
portfolios['promethee'] = PROMETHEE_portfolios
#portfolios['ahp'] = AHP_portfolios

portfolios_stock_reallocation['topsis'] = TOPSIS_portfolios_stock_reallocation
portfolios_stock_reallocation['vikor'] = VIKOR_portfolios_stock_reallocation
portfolios_stock_reallocation['promethee'] = PROMETHEE_portfolios_stock_reallocation
#portfolios_stock_reallocation['ahp'] = AHP_portfolios_stock_reallocation

### COMPUTE VW MCDM STRATEGY
TOPSIS_weighted_portfolios, TOPSIS_weighted_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'topsis', MJ_configuration, weighting=True)
VIKOR_weighted_portfolios, VIKOR_weighted_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'vikor', MJ_configuration, weighting=True)
PROMETHEE_weighted_portfolios, PROMETHEE_weighted_portfolios_stock_reallocation = compute_mcdm_rolling_strategy(df_norm, 'promethee', MJ_configuration, weighting=True)
#AHP_portfolios_weighted, AHP_portfolios_stock_reallocation_weighted = compute_mcdm_rolling_strategy(df_norm, 'ahp', MJ_configuration, weighting=True)


# inserisco VW mcdm in portfolios e reallocation
weighted_portfolios['wtopsis'] = TOPSIS_weighted_portfolios
weighted_portfolios['wvikor'] = VIKOR_weighted_portfolios
weighted_portfolios['wpromethee'] = PROMETHEE_weighted_portfolios
#weighted_portfolios['wahp'] = AHP_portfolios_weighted

portfolios_stock_reallocation['wtopsis'] = TOPSIS_weighted_portfolios_stock_reallocation
portfolios_stock_reallocation['wvikor'] = VIKOR_weighted_portfolios_stock_reallocation
portfolios_stock_reallocation['wpromethee'] = PROMETHEE_weighted_portfolios_stock_reallocation
#portfolios_stock_reallocation['wahp'] = AHP_portfolios_stock_reallocation_weighted


In [ ]:
# ========================== GARA - P10 ==========================

metodi = list(portfolios.keys())                                                    ## Qua modificare per vedere i vari cosi da mettere nella tabella

best_portfolios = {}                                                                                                                   #    <--------------------------------

print("Studio dei rendimenti dei portafogli *port10* (EW) per ogni metodo:")
for metodo in metodi:
    df_metodo = portfolios[metodo]

    if 'port10' not in df_metodo.columns:
        print(f"⚠️ Attenzione: {metodo} non ha port10.")
        continue

    rendimento_port10 = df_metodo['port10'].mean()

    best_portfolios[metodo] = {
        'portafoglio': 'port10',
        'rendimento_medio': rendimento_port10
    }

for metodo, info in best_portfolios.items():
    print(f"{metodo.upper():<10} ➜ rendimento medio mensile EW_P10 = {info['rendimento_medio']:.4f}")

# Trova il metodo il cui port10 ha avuto il rendimento medio più alto
migliore_assoluto = max(best_portfolios.items(), key=lambda x: x[1]['rendimento_medio'])

metodo_top = migliore_assoluto[0]
rendimento_top = migliore_assoluto[1]['rendimento_medio']

print("*" * 50)
print(f"WINNER: Il metodo con il miglior PORT10 è: {metodo_top.upper()} con rendimento medio mensile di {rendimento_top:.4f}")
print("*" * 50)

metodi_w = list(weighted_portfolios.keys())

best_wportfolios = {}                                                                                                                   #    <--------------------------------

print("Studio dei rendimenti dei portafogli *port10* (VW) per ogni metodo:")
for metodo in metodi_w:
    df_metodo = weighted_portfolios[metodo]

    if 'port10' not in df_metodo.columns:
        print(f"⚠️ Attenzione: {metodo} non ha port10.")
        continue

    rendimento_port10 = df_metodo['port10'].mean()

    best_wportfolios[metodo] = {
        'portafoglio': 'port10',
        'rendimento_medio': rendimento_port10
    }

for metodo, info in best_wportfolios.items():
    print(f"{metodo.upper():<10} ➜ rendimento medio mensile VW_P10 = {info['rendimento_medio']:.4f}")

# Trova il metodo il cui port10 ha avuto il rendimento medio più alto
migliore_assoluto = max(best_wportfolios.items(), key=lambda x: x[1]['rendimento_medio'])

metodo_top = migliore_assoluto[0]
rendimento_top = migliore_assoluto[1]['rendimento_medio']

print("*" * 50)
print(f"WINNER: Il metodo con il miglior PORT10 (VW) è: {metodo_top.upper()} con rendimento medio mensile di {rendimento_top:.4f}")
print("*" * 50)


Studio dei rendimenti dei portafogli *port10* (EW) per ogni metodo:
AG         ➜ rendimento medio mensile EW_P10 = 0.0012
BETA       ➜ rendimento medio mensile EW_P10 = 0.0080
BM         ➜ rendimento medio mensile EW_P10 = 0.0180
CUMRET     ➜ rendimento medio mensile EW_P10 = 0.0051
DOLVOL6    ➜ rendimento medio mensile EW_P10 = 0.0064
GP         ➜ rendimento medio mensile EW_P10 = 0.0128
ILL6       ➜ rendimento medio mensile EW_P10 = 0.0181
NS         ➜ rendimento medio mensile EW_P10 = -0.0016
SIZE       ➜ rendimento medio mensile EW_P10 = 0.0071
VOLATILITY ➜ rendimento medio mensile EW_P10 = 0.0057
ACC        ➜ rendimento medio mensile EW_P10 = 0.0070
MJ         ➜ rendimento medio mensile EW_P10 = 0.0194
AHP        ➜ rendimento medio mensile EW_P10 = 0.0054
TOPSIS     ➜ rendimento medio mensile EW_P10 = 0.0185
VIKOR      ➜ rendimento medio mensile EW_P10 = 0.0138
PROMETHEE  ➜ rendimento medio mensile EW_P10 = 0.0191
**************************************************
WINNER: Il metod

In [ ]:
# ========================== GARA - LONG SHORT ==========================

# UGUALE MA CON LONG SHORT
metodi = list(portfolios.keys())

best_portfolios_LS = {}                                                                                                                   #    <--------------------------------

print("Studio dei rendimenti dei portafogli *long_short* (EW) per ogni metodo:")
for metodo in metodi:
    df_metodo = portfolios[metodo]

    if 'long_short' not in df_metodo.columns:
        print(f"⚠️ Attenzione: {metodo} non ha long_short.")
        continue

    rendimento_LS = df_metodo['long_short'].mean()

    best_portfolios_LS[metodo] = {
        'portafoglio': 'long_short',
        'rendimento_medio': rendimento_LS
    }

for metodo, info in best_portfolios_LS.items():
    print(f"{metodo.upper():<10} ➜ rendimento medio mensile EW_LS = {info['rendimento_medio']:.4f}")

# Trova il metodo il cui LS ha avuto il rendimento medio più alto
migliore_assoluto_LS = max(best_portfolios_LS.items(), key=lambda x: x[1]['rendimento_medio'])

metodo_top_LS = migliore_assoluto_LS[0]
rendimento_top_LS = migliore_assoluto_LS[1]['rendimento_medio']

print("*" * 50)
print(f"WINNER: Il metodo con il miglior longshort è: {metodo_top_LS.upper()} con rendimento medio mensile di {rendimento_top_LS:.4f}")
print("*" * 50)

# UGUALE MA CON LONG SHORT
metodi_w = list(weighted_portfolios.keys())

best_wportfolios_WLS = {}                                                                                                                   #    <--------------------------------

print("Studio dei rendimenti dei portafogli *long_short* (VW) per ogni metodo:")
for metodo in metodi_w:
    df_metodo = weighted_portfolios[metodo]

    if 'long_short' not in df_metodo.columns:
        print(f"⚠️ Attenzione: {metodo} non ha long_short.")
        continue

    rendimento_WLS = df_metodo['long_short'].mean()

    best_wportfolios_WLS[metodo] = {
        'portafoglio': 'long_short',
        'rendimento_medio': rendimento_WLS
    }

for metodo, info in best_wportfolios_WLS.items():
    print(f"{metodo.upper():<10} ➜ rendimento medio mensile VW_LS = {info['rendimento_medio']:.4f}")

# Trova il metodo il cui LS ha avuto il rendimento medio più alto
migliore_assoluto_WLS = max(best_wportfolios_WLS.items(), key=lambda x: x[1]['rendimento_medio'])

metodo_top_WLS = migliore_assoluto_WLS[0]
rendimento_top_WLS = migliore_assoluto_WLS[1]['rendimento_medio']

print("*" * 50)
print(f"WINNER: Il metodo con il miglior longshort è: {metodo_top_WLS.upper()} con rendimento medio mensile di {rendimento_top_WLS:.4f}")
print("*" * 50)

Studio dei rendimenti dei portafogli *long_short* (EW) per ogni metodo:
AG         ➜ rendimento medio mensile EW_LS = 0.0115
BETA       ➜ rendimento medio mensile EW_LS = 0.0012
BM         ➜ rendimento medio mensile EW_LS = 0.0114
CUMRET     ➜ rendimento medio mensile EW_LS = -0.0071
DOLVOL6    ➜ rendimento medio mensile EW_LS = 0.0127
GP         ➜ rendimento medio mensile EW_LS = 0.0084
ILL6       ➜ rendimento medio mensile EW_LS = 0.0113
NS         ➜ rendimento medio mensile EW_LS = 0.0143
SIZE       ➜ rendimento medio mensile EW_LS = 0.0095
VOLATILITY ➜ rendimento medio mensile EW_LS = 0.0033
ACC        ➜ rendimento medio mensile EW_LS = 0.0031
MJ         ➜ rendimento medio mensile EW_LS = 0.0169
AHP        ➜ rendimento medio mensile EW_LS = -0.0065
TOPSIS     ➜ rendimento medio mensile EW_LS = 0.0157
VIKOR      ➜ rendimento medio mensile EW_LS = 0.0124
PROMETHEE  ➜ rendimento medio mensile EW_LS = 0.0162
**************************************************
WINNER: Il metodo con il mi

In [ ]:
# ========================== STATISTICHE PORTAFOGLI (TUTTI) ==========================

############################################# P10 ##############################################

def calculate_sharpe_ratio(returns_series, risk_free_rate=0.0):

    '''
    Calcola lo Sharpe Ratio annualizzato.
    Assume returns_series sono rendimenti mensili.
    '''

    mean_return = returns_series.mean() * 12 # Rendimento medio annualizzato
    std_dev = returns_series.std() * np.sqrt(12) # Deviazione standard annualizzata (volatilità)

    annualized_risk_free_rate = risk_free_rate * 12

    if std_dev == 0:
        return np.nan
    return (mean_return - annualized_risk_free_rate) / std_dev

def calculate_sortino_ratio(returns_series, mar_monthly: float = 0.0):
    """
    Sortino Ratio annualizzato.
    - returns_series: rendimenti MENSILI
    - mar_monthly: Minimum Acceptable Return MENSILE (default 0.0). 
      Se vuoi usarlo come risk-free mensile, passalo qui.
    """
    # Numeratore: extra-mean rispetto al MAR, annualizzato
    excess_mean_ann = (returns_series.mean() - mar_monthly) * 12

    # Downside deviation mensile: sqrt( mean( min(0, r - MAR)^2 ) )
    downside_diff = np.minimum(0.0, returns_series - mar_monthly)
    downside_dev_monthly = np.sqrt((downside_diff ** 2).mean())

    # Annualizzazione della downside deviation
    downside_dev_ann = downside_dev_monthly * np.sqrt(12)

    if downside_dev_ann == 0 or np.isnan(downside_dev_ann):
        return np.nan
    return excess_mean_ann / downside_dev_ann

def compute_portfolio_stats(best_portfolios, portfolios_dict, mar_monthly: float = 0.0):
    risultati = []

    for metodo, info in best_portfolios.items():
        portafoglio = info['portafoglio']
        serie = portfolios_dict[metodo][portafoglio]

        rendimento_medio = serie.mean()
        volatilita = serie.std()
        sharpe = calculate_sharpe_ratio(serie)  # usa risk_free mensile = 0.0 di default
        sortino = calculate_sortino_ratio(serie, mar_monthly=mar_monthly)

        # Calcolo drawdown
        cumulative = (1 + serie).cumprod()
        peak = cumulative.cummax()
        drawdown = (cumulative - peak) / peak
        max_drawdown = abs(drawdown.min())

        risultati.append({
            'Metodo': metodo,
            'Portafoglio': portafoglio,
            'Rendimento medio': rendimento_medio,
            'Volatilità': volatilita,
            'Sharpe Ratio': sharpe,
            'Sortino Ratio': sortino,
            'Max Drawdown': max_drawdown
        })

    return pd.DataFrame(risultati)

print("\n--- Statistiche dei portafogli basandosi su P10 ---")
stats_ew = compute_portfolio_stats(best_portfolios, portfolios)
stats_vw = compute_portfolio_stats(best_wportfolios, weighted_portfolios)

print("Statistiche portafogli Equal Weighted (EW):")
print(stats_ew.round(4))
print("\nStatistiche portafogli Value Weighted (VW):")
print(stats_vw.round(4))

print("\n--- Statistiche dei portafogli basandosi su LS ---")
stats_ew_LS = compute_portfolio_stats(best_portfolios_LS, portfolios)
stats_vw_LS = compute_portfolio_stats(best_wportfolios_WLS, weighted_portfolios)

print("Statistiche portafogli Equal Weighted (EW):")
print(stats_ew_LS.round(4))
print("\nStatistiche portafogli Value Weighted (VW):")
print(stats_vw_LS.round(4))


--- Statistiche dei portafogli basandosi su P10 ---
Statistiche portafogli Equal Weighted (EW):
        Metodo Portafoglio  Rendimento medio  Volatilità  Sharpe Ratio  \
0           ag      port10            0.0012      0.0834        0.0519   
1         beta      port10            0.0080      0.1131        0.2448   
2           bm      port10            0.0180      0.0868        0.7180   
3       cumret      port10            0.0051      0.0748        0.2346   
4      dolvol6      port10            0.0064      0.0597        0.3683   
5           gp      port10            0.0128      0.0674        0.6564   
6         ill6      port10            0.0181      0.0776        0.8103   
7           ns      port10           -0.0016      0.0957       -0.0574   
8         size      port10            0.0071      0.0497        0.4976   
9   volatility      port10            0.0057      0.1067        0.1842   
10         acc      port10            0.0070      0.0798        0.3057   
11          mj 

In [ ]:
# ========================== TEST STATISTICI SU SHARPE - BOOTSTRAP A BLOCCHI ==========================

def bootstrap_sharpe_comparison_block(returns1, returns2, num_bootstrap_samples=2000,

    block_size=6, risk_free_rate=0.0):
    """

    Confronta gli Sharpe Ratio di due serie di rendimenti usando il block bootstrap.

    Restituisce la proporzione di volte che Sharpe1 > Sharpe2 (p-value unilaterale).

    Args:

    returns1 (pd.Series): Serie di rendimenti del primo portafoglio.

    returns2 (pd.Series): Serie di rendimenti del secondo portafoglio.

    num_bootstrap_samples (int): Numero di simulazioni bootstrap da eseguire.

    block_size (int): La dimensione del blocco di rendimenti da campionare.

    risk_free_rate (float): Tasso di rendimento risk-free mensile.

    """

    sharpes_diff = []
    n = len(returns1)

    # Calcola il numero di blocchi disponibili
    num_blocks = n // block_size
    if num_blocks == 0:
        print("Errore: la dimensione del blocco è maggiore della lunghezza dei dati.")
        return np.nan


    for _ in range(num_bootstrap_samples):

    # *** MODIFICA: Campionamento con block bootstrap ***
    # 1. Campiona gli INDICI dei blocchi con reinserimento
        block_indices = np.random.choice(num_blocks, size=num_blocks, replace=True)

    # 2. Ricostruisci le serie di rendimenti concatenando i blocchi campionati

        sample1 = np.empty(0)
        sample2 = np.empty(0)
        for idx in block_indices:
            start_idx = idx * block_size
            end_idx = start_idx + block_size
            sample1 = np.concatenate([sample1, returns1.iloc[start_idx:end_idx]])
            sample2 = np.concatenate([sample2, returns2.iloc[start_idx:end_idx]])

        # Assicura che la lunghezza del campione sia esattamente la stessa dell'originale
        sample1 = sample1[:n]
        sample2 = sample2[:n]

        # *** FINE MODIFICA ***

        sharpe1 = calculate_sharpe_ratio(pd.Series(sample1), risk_free_rate)
        sharpe2 = calculate_sharpe_ratio(pd.Series(sample2), risk_free_rate)


        if not np.isnan(sharpe1) and not np.isnan(sharpe2):
            sharpes_diff.append(sharpe1 - sharpe2)


    if not sharpes_diff:
        return np.nan

    p_value_one_sided = np.sum(np.array(sharpes_diff) <= 0) / len(sharpes_diff)

    return p_value_one_sided


# --- Applicazione ---
print("\n--- Confronto Statistico degli Sharpe Ratio basandoci su P10 (WMJ vs Altri VW) ---")
# Tasso risk-free mensile (assumo 0 per semplicità ) --> giustificare durante tesi
RISK_FREE_MONTHLY = 0.0

wmj_port_name = best_wportfolios['wmj']['portafoglio']
wmj_returns_series = weighted_portfolios['wmj'][wmj_port_name]

for metodo in [k for k in weighted_portfolios.keys() if k != 'wmj']:
    other_port_name = best_wportfolios[metodo]['portafoglio']
    other_returns_series = weighted_portfolios[metodo][other_port_name]

    min_len = min(len(wmj_returns_series), len(other_returns_series))
    wmj_sample_returns = wmj_returns_series.iloc[:min_len]
    other_sample_returns = other_returns_series.iloc[:min_len]

    # Chiamo la funzione
    p_value = bootstrap_sharpe_comparison_block(wmj_sample_returns, other_sample_returns, num_bootstrap_samples=2000, risk_free_rate=RISK_FREE_MONTHLY)

    # Interpretazione del p-value   
    alpha = 0.05 # Livello di significatività

    print(f"\nConfronto WMJ vs {metodo.upper()} - P10:")
    if p_value < alpha:
        print(f"La performance risk-adjusted di WMJ è STATISTICAMENTE SUPERIORE a quella di {metodo.upper()} (p-value = {p_value:.4f})")
    else:
        print(f"Non c'è una differenza statisticamente significativa nella performance risk-adjusted tra WMJ e {metodo.upper()} (p-value = {p_value:.4f})")


####################################### ORA UGUALE MA SU LONG-SHORT #############################
# --- Applicazione ---
print("\n--- Confronto Statistico degli Sharpe Ratio basandoci su LS (WMJ vs Altri VW) ---")
# Tasso risk-free mensile (assumo 0 per semplicità ) --> giustificare durante tesi
RISK_FREE_MONTHLY = 0.0

wmj_port_name = best_wportfolios_WLS['wmj']['portafoglio']
wmj_returns_series = weighted_portfolios['wmj'][wmj_port_name]

for metodo in [k for k in weighted_portfolios.keys() if k != 'wmj']:
    other_port_name = best_wportfolios_WLS[metodo]['portafoglio']
    other_returns_series = weighted_portfolios[metodo][other_port_name]

    min_len = min(len(wmj_returns_series), len(other_returns_series))
    wmj_sample_returns = wmj_returns_series.iloc[:min_len]
    other_sample_returns = other_returns_series.iloc[:min_len]

    # Chiamo la funzione
    p_value = bootstrap_sharpe_comparison_block(wmj_sample_returns, other_sample_returns, num_bootstrap_samples=2000, risk_free_rate=RISK_FREE_MONTHLY)

    # Interpretazione del p-value   
    alpha = 0.05 # Livello di significatività

    print(f"\nConfronto WMJ vs {metodo.upper()} - LS:")
    if p_value < alpha:
        print(f"La performance risk-adjusted di WMJ è STATISTICAMENTE SUPERIORE a quella di {metodo.upper()} (p-value = {p_value:.4f})")
    else:
        print(f"Non c'è una differenza statisticamente significativa nella performance risk-adjusted tra WMJ e {metodo.upper()} (p-value = {p_value:.4f})")


# Dai risultati, wmj è tra i portafogli più efficienti (ottimo trade-off rendimento/rischio) 


--- Confronto Statistico degli Sharpe Ratio basandoci su P10 (WMJ vs Altri VW) ---

Confronto WMJ vs AG - P10:
La performance risk-adjusted di WMJ è STATISTICAMENTE SUPERIORE a quella di AG (p-value = 0.0000)

Confronto WMJ vs BETA - P10:
La performance risk-adjusted di WMJ è STATISTICAMENTE SUPERIORE a quella di BETA (p-value = 0.0005)

Confronto WMJ vs BM - P10:
La performance risk-adjusted di WMJ è STATISTICAMENTE SUPERIORE a quella di BM (p-value = 0.0050)

Confronto WMJ vs CUMRET - P10:
La performance risk-adjusted di WMJ è STATISTICAMENTE SUPERIORE a quella di CUMRET (p-value = 0.0005)

Confronto WMJ vs DOLVOL6 - P10:
La performance risk-adjusted di WMJ è STATISTICAMENTE SUPERIORE a quella di DOLVOL6 (p-value = 0.0205)

Confronto WMJ vs GP - P10:
Non c'è una differenza statisticamente significativa nella performance risk-adjusted tra WMJ e GP (p-value = 0.0765)

Confronto WMJ vs ILL6 - P10:
Non c'è una differenza statisticamente significativa nella performance risk-adjusted tra 

In [ ]:
# ========================== TEST STATISTICO SU RENDIMENTI ==========================

from scipy.stats import ttest_rel, wilcoxon, shapiro
from statsmodels.stats.multitest import multipletests
import numpy as np
import pandas as pd

def comprehensive_comparison_test(series_a, series_b, name_a="WMJ", name_b="OTHER", alpha=0.05):
    """
    Esegue test sia parametrici che non-parametrici per confrontare due serie.
    """
    # Allineamento
    idx = series_a.index.intersection(series_b.index)
    a = series_a.loc[idx].dropna()
    b = series_b.loc[idx].dropna()
    idx = a.index.intersection(b.index)
    a = a.loc[idx]
    b = b.loc[idx]
    
    if len(a) < 10:
        return {
            'n_observations': len(a),
            'warning': f"Troppo poche osservazioni ({len(a)})"
        }
    
    # Calcola differenze
    differences = a - b
    
    # Test di normalità
    _, normality_p = shapiro(differences) if len(differences) >= 3 else (None, None)
    is_normal = normality_p > 0.05 if normality_p is not None else False
    
    # T-test parametrico
    t_stat, t_p = ttest_rel(a, b)
    
    # Test non-parametrico di Wilcoxon (più robusto)
    try:
        w_stat, w_p = wilcoxon(differences, alternative='two-sided')
    except:
        w_stat, w_p = np.nan, np.nan
    
    # Statistiche descrittive
    mean_a = a.mean()
    mean_b = b.mean()
    mean_diff = differences.mean()
    median_diff = differences.median()
    std_diff = differences.std()
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt((a.var() + b.var()) / 2)
    cohens_d = mean_diff / pooled_std if pooled_std > 0 else np.nan
    
    return {
        'n_observations': len(a),
        'mean_a': mean_a,
        'mean_b': mean_b,
        'mean_difference': mean_diff,
        'median_difference': median_diff,
        'std_difference': std_diff,
        'cohens_d': cohens_d,
        'normality_p': normality_p,
        'is_normal': is_normal,
        't_statistic': t_stat,
        't_p_value': t_p,
        'wilcoxon_statistic': w_stat,
        'wilcoxon_p_value': w_p,
        't_significant': t_p < alpha,
        'wilcoxon_significant': w_p < alpha,
        'recommended_test': 'wilcoxon' if not is_normal else 't_test'
    }

def perform_multiple_comparisons_analysis(wmj_returns, other_portfolios, alpha=0.05, 
                                        correction_method='holm'):
    """
    Esegue test multipli con correzione per confronti multipli.
    """
    results = {}
    methods = [m for m in other_portfolios.keys() if m != 'wmj']
    
    # Raccogli tutti i p-value
    t_p_values = []
    w_p_values = []
    
    for method_name in methods:
        other_returns = other_portfolios[method_name]
        result = comprehensive_comparison_test(
            wmj_returns, other_returns, "WMJ", method_name.upper(), alpha
        )
        results[method_name] = result
        
        if 'warning' not in result:
            t_p_values.append(result['t_p_value'])
            w_p_values.append(result['wilcoxon_p_value'])
        else:
            t_p_values.append(1.0)  # Non significativo per metodi con warning
            w_p_values.append(1.0)
    
    # Correzione per test multipli
    t_corrected = multipletests(t_p_values, alpha=alpha, method=correction_method)
    w_corrected = multipletests(w_p_values, alpha=alpha, method=correction_method)
    
    # Aggiungi correzioni ai risultati
    for i, method_name in enumerate(methods):
        if 'warning' not in results[method_name]:
            results[method_name]['t_p_corrected'] = t_corrected[1][i]
            results[method_name]['wilcoxon_p_corrected'] = w_corrected[1][i]
            results[method_name]['t_significant_corrected'] = t_corrected[0][i]
            results[method_name]['wilcoxon_significant_corrected'] = w_corrected[0][i]
    
    return results, correction_method

def print_comprehensive_results(results, title, correction_method=None, alpha=0.05):
    """
    Stampa risultati comprensivi con raccomandazioni.
    """
    print(f"\n{'='*80}")
    print(f"{title}")
    print(f"{'='*80}")
    
    # Header
    header = f"{'Metodo':<12} {'n':<4} {'T-test':<8} {'Wilcoxon':<9} {'Raccomandato':<12} {'Effect Size':<11} {'Note'}"
    print(header)
    print("-" * len(header))
    
    significant_methods = []
    
    for method, result in results.items():
        if 'warning' in result:
            print(f"{method.upper():<12} {result['n_observations']:<4} {'N/A':<8} {'N/A':<9} {'N/A':<12} {'N/A':<11} {result['warning']}")
            continue
        
        method_label = method.upper()
        
        # Scegli il test raccomandato
        if result['recommended_test'] == 'wilcoxon':
            main_p = result['wilcoxon_p_corrected'] if correction_method else result['wilcoxon_p_value']
            is_significant = main_p < alpha
            test_symbol = "W" if is_significant else "w"
        else:
            main_p = result['t_p_corrected'] if correction_method else result['t_p_value']
            is_significant = main_p < alpha
            test_symbol = "T" if is_significant else "t"
        
        # Effect size interpretation
        d = abs(result['cohens_d'])
        if d >= 0.8:
            effect = "Grande"
        elif d >= 0.5:
            effect = "Medio"  
        elif d >= 0.2:
            effect = "Piccolo"
        else:
            effect = "Trascurabile"
        
        # Note
        note = ""
        if not result['is_normal']:
            note += "Non-norm"
        if result['mean_difference'] > 0:
            note += " WMJ>" + method_label[:3]
        else:
            note += " WMJ<" + method_label[:3]
        
        t_p_display = f"{result['t_p_value']:.4f}"
        w_p_display = f"{result['wilcoxon_p_value']:.4f}"
        sig_symbol = "✅" if is_significant else "➖"
        
        print(f"{method_label:<12} {result['n_observations']:<4} {t_p_display:<8} {w_p_display:<9} "
              f"{test_symbol+sig_symbol:<12} {effect:<11} {note}")
        
        if is_significant:
            significant_methods.append(method_label)
    
    print(f"\n🎯 WMJ significativamente superiore a: {significant_methods}")
    print(f"📊 Test raccomandato: Wilcoxon (dati non-normali)")
    if correction_method:
        print(f"🔧 Correzione applicata: {correction_method}")

# ==================== APPLICAZIONE ======================

alpha = 0.05

# P10 Analysis
print("ANALISI STATISTICA ROBUSTA: P10 VW")
wmj_ret_p10 = weighted_portfolios['wmj'][best_wportfolios['wmj']['portafoglio']]
other_portfolios_p10 = {
    method: weighted_portfolios[method][best_wportfolios[method]['portafoglio']]
    for method in weighted_portfolios.keys() if method != 'wmj'
}

results_p10, correction_method = perform_multiple_comparisons_analysis(
    wmj_ret_p10, other_portfolios_p10, alpha, 'holm'
)
print_comprehensive_results(results_p10, "P10 VW: WMJ vs Altri Metodi (Test Robusti)", correction_method, alpha)

# LS Analysis  
print("\n\nANALISI STATISTICA ROBUSTA: LS VW")
wmj_ret_ls = weighted_portfolios['wmj'][best_wportfolios_WLS['wmj']['portafoglio']]
other_portfolios_ls = {
    method: weighted_portfolios[method][best_wportfolios_WLS[method]['portafoglio']]
    for method in weighted_portfolios.keys() if method != 'wmj'
}

results_ls, correction_method = perform_multiple_comparisons_analysis(
    wmj_ret_ls, other_portfolios_ls, alpha, 'holm'
)
print_comprehensive_results(results_ls, "LS VW: WMJ vs Altri Metodi (Test Robusti)", correction_method, alpha)

# Summary insight
print(f"\n{'='*80}")
print("📈 RIEPILOGO INSIGHTS")
print(f"{'='*80}")
print("• I rendimenti finanziari raramente seguono distribuzione normale")
print("• Il test di Wilcoxon è più appropriato per questi dati") 
print("• La correzione di Holm controlla il tasso di errore di Tipo I nei test multipli")
print("• L'effect size (Cohen's d) quantifica la dimensione pratica delle differenze")
print("• WMJ sembra performare meglio nella strategia P10 che in Long-Short")

ANALISI STATISTICA ROBUSTA: P10 VW

P10 VW: WMJ vs Altri Metodi (Test Robusti)
Metodo       n    T-test   Wilcoxon  Raccomandato Effect Size Note
------------------------------------------------------------------
AG           282  0.0000   0.0001    W✅           Trascurabile Non-norm WMJ>AG
BETA         282  0.1010   0.0682    w➖           Trascurabile Non-norm WMJ>BET
BM           282  0.1251   0.0701    w➖           Trascurabile Non-norm WMJ>BM
CUMRET       282  0.0050   0.0027    W✅           Trascurabile Non-norm WMJ>CUM
DOLVOL6      282  0.0003   0.0017    W✅           Trascurabile Non-norm WMJ>DOL
GP           282  0.0031   0.0067    T✅           Trascurabile  WMJ>GP
ILL6         282  0.2275   0.2811    w➖           Trascurabile Non-norm WMJ>ILL
NS           282  0.0000   0.0000    W✅           Piccolo     Non-norm WMJ>NS
SIZE         282  0.0004   0.0022    W✅           Trascurabile Non-norm WMJ>SIZ
VOLATILITY   282  0.0124   0.0048    W✅           Trascurabile Non-norm WMJ>VOL


In [ ]:
# ========================== COSTI DI TRANSAZIONE ==========================

def comprehensive_transaction_cost_analysis_extended():
    """
    Analisi completa dei costi di transazione per P10 e Long-Short
    """
    
    # 1. Raccogli turnover e rendimenti per P10 VW
    turnover_data_p10 = {}
    gross_returns_p10 = {}
    
    for method in weighted_portfolios.keys():
        if method in portfolios_stock_reallocation:
            df_turn = portfolios_stock_reallocation[method]['VW_turnover']
            turnover_p10 = df_turn[df_turn['portfolio'] == 10.0]['VW_turnover'].values[0]
            turnover_data_p10[method] = turnover_p10
            gross_returns_p10[method] = weighted_portfolios[method]['port10'].mean()
    
    # 2. Raccogli turnover e rendimenti per Long-Short VW
    # Per LS assumiamo che il turnover sia la media tra P1 e P10 (o usa metrica specifica se disponibile)
    turnover_data_ls = {}
    gross_returns_ls = {}
    
    for method in weighted_portfolios.keys():
        if method in portfolios_stock_reallocation:
            df_turn = portfolios_stock_reallocation[method]['VW_turnover']
            # Turnover LS = media ponderata tra P1 e P10 (o puoi usare metrica diversa)
            turnover_p1 = df_turn[df_turn['portfolio'] == 1.0]['VW_turnover'].values[0]
            turnover_p10 = df_turn[df_turn['portfolio'] == 10.0]['VW_turnover'].values[0]
            turnover_data_ls[method] = (turnover_p1 + turnover_p10) / 2  # Media semplice
            gross_returns_ls[method] = weighted_portfolios[method]['long_short'].mean()
    
    # 3. Range di costi di transazione realistici
    transaction_costs = [0.001, 0.002, 0.005, 0.01, 0.015, 0.02]  # 0.1% - 2%
    
    # 4. Calcola rendimenti netti per P10
    results_p10 = []
    for cost in transaction_costs:
        for method in turnover_data_p10.keys():
            gross_ret = gross_returns_p10[method]
            turnover = turnover_data_p10[method]
            
            cost_impact = turnover * cost
            net_return = gross_ret - cost_impact
            
            gross_series = weighted_portfolios[method]['port10']
            net_series = gross_series - cost_impact
            net_sharpe = calculate_sharpe_ratio(net_series)
            
            results_p10.append({
                'Strategy': 'P10',
                'Method': method.upper(),
                'Transaction_Cost': cost,
                'Turnover': turnover,
                'Gross_Return': gross_ret,
                'Cost_Impact': cost_impact,
                'Net_Return': net_return,
                'Net_Sharpe': net_sharpe,
                'Return_Loss_pct': (cost_impact / gross_ret) * 100 if gross_ret != 0 else 0
            })
    
    # 5. Calcola rendimenti netti per Long-Short
    results_ls = []
    for cost in transaction_costs:
        for method in turnover_data_ls.keys():
            gross_ret = gross_returns_ls[method]
            turnover = turnover_data_ls[method]
            
            cost_impact = turnover * cost
            net_return = gross_ret - cost_impact
            
            gross_series = weighted_portfolios[method]['long_short']
            net_series = gross_series - cost_impact
            net_sharpe = calculate_sharpe_ratio(net_series)
            
            results_ls.append({
                'Strategy': 'Long-Short',
                'Method': method.upper(),
                'Transaction_Cost': cost,
                'Turnover': turnover,
                'Gross_Return': gross_ret,
                'Cost_Impact': cost_impact,
                'Net_Return': net_return,
                'Net_Sharpe': net_sharpe,
                'Return_Loss_pct': (cost_impact / gross_ret) * 100 if gross_ret != 0 else 0
            })
    
    # 6. Combina i risultati
    all_results = results_p10 + results_ls
    return pd.DataFrame(all_results)

# Esegui analisi estesa
cost_analysis_extended = comprehensive_transaction_cost_analysis_extended()

# 7. Ranking separato per P10 e Long-Short
def print_strategy_rankings(df, strategy_name, costs_to_show=[0.001, 0.005, 0.01, 0.02]):
    print(f"\n🏆 RANKING {strategy_name.upper()} - RENDIMENTO NETTO:")
    print("=" * 80)
    
    strategy_data = df[df['Strategy'] == strategy_name]
    
    for cost in costs_to_show:
        subset = strategy_data[strategy_data['Transaction_Cost'] == cost].copy()
        subset = subset.sort_values('Net_Return', ascending=False)
        
        print(f"\n📊 Costo transazione: {cost*100:.1f}%")
        print(f"{'Rank':<4} {'Method':<12} {'Net Return':<12} {'Turnover':<10} {'Loss %':<8}")
        print("-" * 50)
        
        for i, (_, row) in enumerate(subset.head(5).iterrows(), 1):
            print(f"{i:<4} {row['Method']:<12} {row['Net_Return']:.4f}      {row['Turnover']:.4f}    {row['Return_Loss_pct']:.2f}%")
        
        # Posizione di WMJ
        wmj_subset = subset[subset['Method'] == 'WMJ']
        if not wmj_subset.empty:
            wmj_pos = subset.reset_index().query("Method == 'WMJ'").index[0] + 1
            print(f"🎯 WMJ Position: #{wmj_pos}")

# Stampa ranking per entrambe le strategie
print_strategy_rankings(cost_analysis_extended, 'P10')
print_strategy_rankings(cost_analysis_extended, 'Long-Short')

# 8. Analisi break-even comparativa
def break_even_analysis(df, strategy_name):
    print(f"\n\n💰 BREAK-EVEN ANALYSIS - {strategy_name.upper()}:")
    print("=" * 50)
    
    strategy_data = df[df['Strategy'] == strategy_name]
    wmj_data = strategy_data[strategy_data['Method'] == 'WMJ']
    
    if wmj_data.empty:
        print(f"⚠️ WMJ non trovato per {strategy_name}")
        return
    
    wmj_gross = wmj_data['Gross_Return'].iloc[0]
    wmj_turnover = wmj_data['Turnover'].iloc[0]
    
    print(f"WMJ Turnover: {wmj_turnover:.4f}")
    print(f"WMJ Gross Return: {wmj_gross:.4f}")
    
    competitors = ['WTOPSIS', 'WVIKOR', 'WPROMETHEE', 'DOLVOL6', 'GP', 'ILL6']
    unique_methods = strategy_data['Method'].unique()
    
    for comp in competitors:
        if comp in unique_methods:
            comp_data = strategy_data[strategy_data['Method'] == comp]
            comp_gross = comp_data['Gross_Return'].iloc[0]
            comp_turnover = comp_data['Turnover'].iloc[0]
            
            if abs(comp_turnover - wmj_turnover) > 1e-6:
                breakeven_cost = (comp_gross - wmj_gross) / (comp_turnover - wmj_turnover)
                if breakeven_cost > 0:
                    print(f"vs {comp}: Break-even cost = {breakeven_cost*100:.2f}%")
                else:
                    print(f"vs {comp}: WMJ dominante (sempre superiore)")

# Esegui break-even per entrambe
break_even_analysis(cost_analysis_extended, 'P10')
break_even_analysis(cost_analysis_extended, 'Long-Short')

# 10. Tabelle finali comparative
def create_summary_table(df, strategy_name):
    strategy_data = df[df['Strategy'] == strategy_name]
    summary_costs = strategy_data[strategy_data['Transaction_Cost'].isin([0.001, 0.005, 0.01])]
    
    pivot_table = summary_costs.pivot(index='Method', columns='Transaction_Cost', values='Net_Return')
    pivot_table.columns = [f'Cost_{c*100:.1f}%' for c in pivot_table.columns]
    pivot_table = pivot_table.sort_values('Cost_0.5%', ascending=False)
    
    print(f"\n📋 SUMMARY TABLE - {strategy_name.upper()} Net Returns by Transaction Cost:")
    print(pivot_table.round(4))
    
    return pivot_table

# Crea tabelle per entrambe le strategie
summary_p10 = create_summary_table(cost_analysis_extended, 'P10')
summary_ls = create_summary_table(cost_analysis_extended, 'Long-Short')

# 11. Confronto diretto P10 vs Long-Short per WMJ
print("\n🔍 WMJ: P10 vs Long-Short Performance")
print("=" * 50)

wmj_comparison = cost_analysis_extended[cost_analysis_extended['Method'] == 'WMJ']
for cost in [0.001, 0.005, 0.01, 0.02]:
    cost_data = wmj_comparison[wmj_comparison['Transaction_Cost'] == cost]
    p10_row = cost_data[cost_data['Strategy'] == 'P10']
    ls_row = cost_data[cost_data['Strategy'] == 'Long-Short']
    
    if not p10_row.empty and not ls_row.empty:
        p10_net = p10_row['Net_Return'].iloc[0]
        ls_net = ls_row['Net_Return'].iloc[0]
        p10_turnover = p10_row['Turnover'].iloc[0]
        ls_turnover = ls_row['Turnover'].iloc[0]
        
        print(f"\nCosto {cost*100:.1f}%:")
        print(f"  P10:        Net Return = {p10_net:.4f}, Turnover = {p10_turnover:.4f}")
        print(f"  Long-Short: Net Return = {ls_net:.4f}, Turnover = {ls_turnover:.4f}")
        print(f"  Differenza: {ls_net - p10_net:.4f} {'(LS > P10)' if ls_net > p10_net else '(P10 > LS)'}")


🏆 RANKING P10 - RENDIMENTO NETTO:

📊 Costo transazione: 0.1%
Rank Method       Net Return   Turnover   Loss %  
--------------------------------------------------
1    WMJ          0.0142      0.3452    2.37%
2    ILL6         0.0122      0.2172    1.74%
3    WPROMETHEE   0.0118      0.5511    4.47%
4    WTOPSIS      0.0108      0.5614    4.93%
5    BM           0.0102      0.2378    2.28%
🎯 WMJ Position: #1

📊 Costo transazione: 0.5%
Rank Method       Net Return   Turnover   Loss %  
--------------------------------------------------
1    WMJ          0.0128      0.3452    11.87%
2    ILL6         0.0114      0.2172    8.72%
3    WPROMETHEE   0.0096      0.5511    22.37%
4    BM           0.0093      0.2378    11.39%
5    WTOPSIS      0.0086      0.5614    24.66%
🎯 WMJ Position: #1

📊 Costo transazione: 1.0%
Rank Method       Net Return   Turnover   Loss %  
--------------------------------------------------
1    WMJ          0.0111      0.3452    23.74%
2    ILL6         0.0103     